# <div style="text-align: center"><font color='#dc2624' face='微软雅黑'>PyEcharts</font></div>

## <font color='#dc2624' face='微软雅黑'>目录</font><a name='toc'></a>
### 1. [**<font color='#dc2624' face='微软雅黑'>PyEcharts 101</font>**](#1)

### 2. [**<font color='#dc2624' face='微软雅黑'>坐标系图</font>**](#2)
1. [<font color='#2b4750' face='微软雅黑'>K 线图</font>](#2.1)
2. [<font color='#2b4750' face='微软雅黑'>折线图</font>](#2.2)

### 3. [**<font color='#dc2624' face='微软雅黑'>非坐标系图</font>**](#3)
1. [<font color='#2b4750' face='微软雅黑'>词云图</font>](#3.1)
2. [<font color='#2b4750' face='微软雅黑'>树形图</font>](#3.2)

### 4. [**<font color='#dc2624' face='微软雅黑'>多图</font>**](#4)
1. [<font color='#2b4750' face='微软雅黑'>并行图</font>](#4.1)
2. [<font color='#2b4750' face='微软雅黑'>时间轮播图</font>](#4.2)
3. [<font color='#2b4750' face='微软雅黑'>选项卡图</font>](#4.3)
4. [<font color='#2b4750' face='微软雅黑'>页面图</font>](#4.4)

<div style="text-align: right"> Use a picture. It’s worth a thousand words.</div>
<div style="text-align: right"> — Arthur Brisbane</div>

# <font color='#dc2624' face='微软雅黑'>1. `PyEcharts` 101</font><a name='1'></a>
[<font color='black' face='微软雅黑'>回到目录</font>](#toc)

**`PyEcharts`** 顾名思义就是 Python 和 Echarts 的合体。Echarts 是一个由百度开源的数据可视化工具，凭借着良好的交互性，精巧的图表设计，得到了众多开发者的认可，而 Python 就不用多说了。当 Python 遇到了 Echarts，就变成了 **`PyEcharts`**。

**`PyEcharts`** 的官网链接是 https://pyecharts.org。

在使用 **`PyEcharts`** 之前，需要引进它并检查它的版本，语法如下：

In [4]:
import pyecharts
pyecharts.__version__

'1.9.0'

In [5]:
import numpy as np
import pandas as pd
from datetime import datetime

在 **`PyEcharts`** 中，一切皆配置 (options)。配置设定得越细就能画出越多细节。配置项有两种：22 个全局配置项和 17 个系列配置项。在后面会重点说明，尤其是全局配置项，它可通过 set_global_options() 方法来设置。借用其官网上的图来显示 6 个常见全局配置项。

<img src='options.png' style="width:80%; height:80%;">

在 **`PyEcharts`** 中图表可分两大类，单图和多图。

- **单图**包括常见柱状和线形图等之外，还有“罕见”的 K 线图 (kline)、树形图 (treemap) 和词云图 (wordcount)

- **多图**包括并行 (grid)、时间轮播 (timeline)、选项卡 (tab) 和顺序 (page) 四小类

在 v1.0 版本中，从 `pyecharts.charts` 中引入绘图元件。

In [6]:
from pyecharts.charts import Line, Kline, Bar, Pie, WordCloud, TreeMap
from pyecharts.charts import Grid, Timeline, Page, Tab
from pyecharts import options as opts
from pyecharts.globals import SymbolType
from pyecharts.commons.utils import JsCode

前两行分别引入单图原件和多图原件、第三行引入配置模块，第四行引入的 SymbolType 用于设置词云的形状，第五行引入的 JsCode 用于运行 JavaScript 语句，在后面格式化字符串会用到。

# <font color='#dc2624' face='微软雅黑'>2. 坐标系图</font><a name='2'></a>
[<font color='black' face='微软雅黑'>回到目录</font>](#toc)

首先用 YahooFinancials API 来下载若干资产的一年历史数据 (安装该 API 用 pip install yahoofinancials)：

- 起始日：2018-12-01
- 终止日：2020-12-01
- 两只股票：蔚来 (NIO)、特斯拉 (TSLA)
- 三个汇率：欧元美元 (EURUSD)、美元日元 (USDJPY)、英镑美元 (GBPUSD)
- 两个加密货币：比特币 (BTC)、以太坊币 (ETH)

下面代码就是从 API 获取数据，股票用的是股票代号 (stock code)，而货币用的该 API 要求的格式，比如「欧元美元」用 EURUSD=X，而不是市场常见的 EURUSD，而「美元日元」用 JPY=X 而不是 USDJPY。

In [7]:
from yahoofinancials import YahooFinancials

In [8]:
start_date = '2018-12-01'
end_date = '2020-12-01'

stock_code = ['NIO', 'TSLA']
currency_code = ['EURUSD=X', 'JPY=X', 'GBPUSD=X']
cryptocurrency_code = ['BTC-USD', 'ETH-USD']

stock = YahooFinancials( stock_code )
currency = YahooFinancials( currency_code )
cryptocurrency = YahooFinancials( cryptocurrency_code )

stock_daily = stock.get_historical_price_data( start_date, end_date, 'daily' )
currency_daily = currency.get_historical_price_data( start_date, end_date, 'daily' )
cryptocurrency_daily = cryptocurrency.get_historical_price_data( start_date, end_date, 'daily' )

该 API 返回结果是字典格式，样子非常丑陋，但信息齐全。画 K 线需要的开盘价 (open)、最高价 (high)、最低价 (low)、收盘价 (close) 都有。将上面的原始数据转换成数据帧，代码如下：

In [9]:
def data_converter( price_data, code, asset ):

    if asset == 'FX':
        code = str(code[3:] if code[:3]=='USD' else code) + '=X' 

    columns = ['open', 'close', 'low', 'high' ]
    price_dict = price_data[code]['prices']
    index = [ p['formatted_date'] for p in price_dict ]
    price = [ [p[c] for c in columns] for p in price_dict ]

    data = pd.DataFrame( price,
                         index=pd.Index(index, name='date'),
                         columns=pd.Index(columns, name='OHLC') )
    return data

### <font color='#2b4750' face='微软雅黑'>2.1 K 线图</font><a name='2.1'></a>
[<font color='black' face='微软雅黑'>回到章首</font>](#2)

In [10]:
stock = 'NIO'
data = data_converter( stock_daily, stock, 'EQ' )
data.head(3).append(data.tail(3))

OHLC,open,close,low,high
date,,,,
2018-12-03,8.100000,7.580000,7.500000,8.130000
2018-12-04,7.600000,7.080000,7.050000,7.640000
2018-12-06,6.860000,7.370000,6.720000,7.370000
2020-11-25,49.980000,53.689999,49.250000,53.990002
2020-11-27,54.860001,54.000000,52.599998,55.549999
2020-11-30,54.209999,50.529999,48.180000,54.389999


In [22]:
date, price = data.index, data.values
date_list = pd.to_datetime(date).strftime('%Y/%m/%d').tolist()

kline = (
    Kline()
    .add_xaxis(date_list)
    .add_yaxis('', price.tolist())
    .set_global_opts(
        xaxis_opts=opts.AxisOpts(is_scale=True),
        yaxis_opts=opts.AxisOpts(
            is_scale=True,
            splitarea_opts=opts.SplitAreaOpts(
                is_show=True, 
                areastyle_opts=opts.AreaStyleOpts(opacity=0.5)
            ),
        ),
        title_opts=opts.TitleOpts(title=stock +' Chart'),
        datazoom_opts=[opts.DataZoomOpts(type_='inside')],
    )
)

kline.render_notebook()

在网页中展示可用 `render('xxx.html')` 语句。

In [23]:
kline.render(path=u'NIO Chart (v1.9.0).html');

在 K 线图中添加折线，用 `overlap()` 函数。

In [27]:
kline = (
    Kline()
    .add_xaxis(date_list)
    .add_yaxis("K Line", price.tolist())
    .set_global_opts(
        xaxis_opts=opts.AxisOpts(is_scale=True),
        yaxis_opts=opts.AxisOpts(
            is_scale=True,
            splitarea_opts=opts.SplitAreaOpts(
                is_show=True, 
                areastyle_opts=opts.AreaStyleOpts(opacity=1)
            ),
        ),
        title_opts=opts.TitleOpts(title=stock+' Chart'),
        datazoom_opts=[opts.DataZoomOpts(type_='inside'),
                       opts.DataZoomOpts(type_='slider')],
    )
)

line = (
    Line()
    .add_xaxis(date_list)
    .add_yaxis(
            series_name='high',
            y_axis=data['high'].tolist(),
            is_smooth=True,
            is_hover_animation=False,
            linestyle_opts=opts.LineStyleOpts(width=1, opacity=0.5),
            label_opts=opts.LabelOpts(is_show=False),
    )
    .add_yaxis(
            series_name='low',
            y_axis=data['low'].tolist(),
            is_smooth=True,
            is_hover_animation=False,
            linestyle_opts=opts.LineStyleOpts(width=1, opacity=0.5),
            label_opts=opts.LabelOpts(is_show=False),
    )
    .set_global_opts(datazoom_opts=[opts.DataZoomOpts(type_='inside'),
                                    opts.DataZoomOpts(type_='slider')] )
)
    
overlap_kline_line = kline.overlap(line)
overlap_kline_line.render_notebook()

In [28]:
overlap_kline_line.render(path=u'NIO Chart (v1.9.0).html');

### <font color='#2b4750' face='微软雅黑'>2.2 折线图</font><a name='2.2'></a>
[<font color='black' face='微软雅黑'>回到章首</font>](#2)

In [29]:
label = 'BTC-USD'
data = data_converter( cryptocurrency_daily, label, 'CFX' )
data.head(3).append(data.tail(3))

OHLC,open,close,low,high
date,,,,
2018-12-01,4024.464355,4214.671875,3969.710693,4309.377441
2018-12-02,4200.733398,4139.877930,4110.978516,4301.519531
2018-12-03,4147.323730,3894.130859,3840.446289,4155.979492
2020-11-29,17719.634766,18177.484375,17559.117188,18283.628906
2020-11-30,18178.322266,19625.835938,18178.322266,19749.263672
2020-12-01,19633.769531,18802.998047,18321.921875,19845.974609


In [31]:
#df = data['2018':'2019']
date, close = data.index, data['close'].values
date_list = pd.to_datetime(date).strftime('%Y/%m/%d').tolist()

lines = (
    Line()
    .add_xaxis(date_list)
    .add_yaxis(
            series_name='close',
            y_axis=close,
            is_smooth=True,
            is_hover_animation=False,
            linestyle_opts=opts.LineStyleOpts(width=2, opacity=0.5),
            label_opts=opts.LabelOpts(is_show=False),
    )
    .set_global_opts(title_opts=opts.TitleOpts(title='比特币价格图'),
                     datazoom_opts=[opts.DataZoomOpts(type_='inside'),
                                    opts.DataZoomOpts(type_='slider')])
)

window_periods = [5, 20, 60]
for wp in window_periods:
    MA = data['close'].rolling(window=wp).mean()
    lines.add_yaxis(
        series_name='MA'+str(wp),
        y_axis=MA,
        is_smooth=True,
        is_hover_animation=False,
        linestyle_opts=opts.LineStyleOpts(width=1),
        label_opts=opts.LabelOpts(is_show=False),
    )

lines.render_notebook()

# <font color='#dc2624' face='微软雅黑'>3. 非坐标系图</font><a name='3'></a>
[<font color='black' face='微软雅黑'>回到目录</font>](#toc)

### <font color='#2b4750' face='微软雅黑'>3.1 词云图</font><a name='3.1'></a>
[<font color='black' face='微软雅黑'>回到章首</font>](#3)

词云图是对文本中出现频率较高的“关键词”做可视化，它过滤掉低频低质的文本信息，使得浏览者一眼扫过文本就可领略主旨。以下用笔者个人公众号「王的机器」用户分布信息来做词云图，首先收集地区名称和属于该地区的用户个数。

In [32]:
attr = ['北京', '上海', '深圳', '广州', '杭州', '美国', '南京', '成都', '武汉', '西安', 
        '新加坡', '重庆', '长沙', '天津', '苏州', '合肥', '郑州', '厦门', '香港',  '济南', 
        '大连', '青岛', '佛山', '沈阳', '福州', '东莞', '哈尔滨', '加拿大', '昆明', '南昌', 
        '秦皇岛', '宁波', '珠海', '澳大利亚', '长春', '兰州', '无锡', '南宁', '烟台', '石家庄', 
        '英国', '日本', '太原', '德国', '贵阳', '徐州', '温州', '呼和浩特', '保定', '嘉兴']
count = [5209, 4314, 2247, 1503, 1399, 1028, 927, 904, 845, 796,
         654, 410, 405, 402, 333, 314, 292, 263, 245, 242,
         239, 225, 212, 210, 196, 178, 175, 164, 152, 151,
         145, 144, 140, 133, 129, 128, 120, 120, 109,
         104, 104, 98, 91, 87,86, 74, 65, 62, 60]

In [36]:
wordcloud = (
    WordCloud()
    .add(series_name="",
         data_pair=[pair for pair in zip(attr,count)], 
         word_size_range=[20, 100]
        )
    .set_global_opts(
        title_opts=opts.TitleOpts(
            title="王的机器用户分析", title_textstyle_opts=opts.TextStyleOpts(font_size=23)
        ),
        tooltip_opts=opts.TooltipOpts(is_show=True),
    )
)

wordcloud.render_notebook()

In [40]:
wordcloud = (
    WordCloud()
    .add(series_name="",
         data_pair=[pair for pair in zip(attr,count)], 
         word_size_range=[20, 100],
         shape=SymbolType.DIAMOND)
    .set_global_opts(
        title_opts=opts.TitleOpts(
            title="王的机器用户分析", title_textstyle_opts=opts.TextStyleOpts(font_size=23)
        ),
        tooltip_opts=opts.TooltipOpts(is_show=True),
    )
)

wordcloud.render_notebook()

### <font color='#2b4750' face='微软雅黑'>3.2 树形图</font><a name='3.2'></a>
[<font color='black' face='微软雅黑'>回到章首</font>](#3)

树形图可在嵌套的且不同大小和颜色矩形中显示数据，数据可以用层级方式结构化。用 `Pandas` 读取并打印数据。

数据帧中四列分别是美股的股票代号 (code)、市值 (market_cap)、日收益率 (return) 和行业 (sector)。

In [46]:
data = pd.read_csv( 'US Stock Info.csv' )
data.head()

,code,market_cap,return,sector
0,AAPL,8.860000e+11,0.012412,TECHNOLOGY
1,ABT,1.430000e+11,0.006442,HEALTHCARE
2,GOLD,2.360000e+10,-0.006642,BASIC_MATERIALS
3,ADSK,3.630000e+10,-0.001089,TECHNOLOGY
4,ADBE,1.370000e+11,0.007730,TECHNOLOGY


In [47]:
def print_groups( group_obj ):
    for name, group in group_obj:
        print( name )
        print( group.head() )

In [48]:
grouped = data.groupby(['sector'])
print_groups(grouped)

BASIC_MATERIALS
     code    market_cap    return           sector
2    GOLD  2.360000e+10 -0.006642  BASIC_MATERIALS
16    APD  4.870000e+10 -0.007310  BASIC_MATERIALS
55    ECL  5.760000e+10  0.004327  BASIC_MATERIALS
115   NEM  2.890000e+10 -0.011208  BASIC_MATERIALS
133   PPG  2.700000e+10  0.003070  BASIC_MATERIALS
COMMUNICATION_SERVICES
      code    market_cap    return                  sector
25     BCE  4.210000e+10 -0.001067  COMMUNICATION_SERVICES
42   CMCSA  1.870000e+11 -0.004099  COMMUNICATION_SERVICES
141      T  2.330000e+11 -0.017549  COMMUNICATION_SERVICES
195   DISH  1.750000e+10  0.005925  COMMUNICATION_SERVICES
201    RCI  2.740000e+10  0.000000  COMMUNICATION_SERVICES
CONSUMER_CYCLICAL
   code    market_cap    return             sector
20  AZO  2.730000e+10  0.011951  CONSUMER_CYCLICAL
24  BBY  1.740000e+10  0.022599  CONSUMER_CYCLICAL
29  BLL  2.220000e+10  0.006373  CONSUMER_CYCLICAL
52  DIS  2.470000e+11 -0.006956  CONSUMER_CYCLICAL
62    F  3.920000e+10  0.005

In [49]:
group_value = grouped.apply( lambda df: np.sum(df['market_cap']) )
group_value

sector
BASIC_MATERIALS           3.367000e+11
COMMUNICATION_SERVICES    1.067600e+12
CONSUMER_CYCLICAL         2.946600e+12
CONSUMER_DEFENSIVE        2.150900e+12
ENERGY                    1.270500e+12
FINANCIAL_SERVICES        4.461700e+12
HEALTHCARE                3.076000e+12
INDUSTRIALS               2.365500e+12
REAL_ESTATE               5.053000e+11
TECHNOLOGY                5.941700e+12
UTILITIES                 6.931000e+11
dtype: float64

接下来编写一个函数将数据帧转换成 **`PyEcharts`** 中树形图需要的数据格式。

In [50]:
data_for_treemap = []

for i, grp in enumerate(grouped):
    
    total_mktcap, sector = group_value[i], group_value.index[i]
    code, r, mktcap = grp[1]['code'], grp[1]['return'], grp[1]['market_cap']

    children = []
    
    for j in range(len(code)):
        
        j_data = { 'value' : mktcap.iloc[j], 
                   'name': [code.iloc[j], r.iloc[j], sector] }
        
        children.append( j_data )
    
    i_data = { 'value' : total_mktcap,
               'name': sector,
               'children': children }
    
    data_for_treemap.append( i_data )

data_for_treemap

[{'value': 336700000000.0,
  'name': 'BASIC_MATERIALS',
  'children': [{'value': 23600000000.0,
    'name': ['GOLD', -0.006642, 'BASIC_MATERIALS']},
   {'value': 48700000000.0, 'name': ['APD', -0.00731, 'BASIC_MATERIALS']},
   {'value': 57600000000.0, 'name': ['ECL', 0.004327, 'BASIC_MATERIALS']},
   {'value': 28900000000.0, 'name': ['NEM', -0.011208, 'BASIC_MATERIALS']},
   {'value': 27000000000.0, 'name': ['PPG', 0.00307, 'BASIC_MATERIALS']},
   {'value': 43500000000.0, 'name': ['SHW', -0.004979, 'BASIC_MATERIALS']},
   {'value': 17900000000.0, 'name': ['VMC', 0.009007, 'BASIC_MATERIALS']},
   {'value': 28100000000.0, 'name': ['SCCO', 0.009154, 'BASIC_MATERIALS']},
   {'value': 31000000000.0, 'name': ['LYB', 0.056313, 'BASIC_MATERIALS']},
   {'value': 30400000000.0, 'name': ['NTR', 0.004647, 'BASIC_MATERIALS']}]},
 {'value': 1067600000000.0,
  'name': 'COMMUNICATION_SERVICES',
  'children': [{'value': 42100000000.0,
    'name': ['BCE', -0.001067, 'COMMUNICATION_SERVICES']},
   {'valu

In [56]:
label_formatter = """function (params) {
        sign = (params.name[1] > 0) ? '+' : '';
        return params.name[0] + '\\n' + sign + window.parseFloat(params.name[1]*100).toFixed(2) + '%'
    }"""

In [52]:
tooltip_formatter = """function (params) {
up_down_flat = (params.name[1] > 0) ? '涨' : (params.name[1] < 0) ? '跌' : '平';
return params.name[0] + '<br>' + '行业: ' + params.name[2]  + 
        '<br>' + '市值: ' + window.parseFloat(params.value/100000000) + ' 亿美元' +
        '<br>' + '涨跌: ' + up_down_flat;
    }"""

In [58]:
treemap = (
    TreeMap( init_opts=opts.InitOpts(width="1000px", height="600px") )
    .add("", 
         data_for_treemap,
         label_opts=opts.LabelOpts(position="inside", font_size=10, formatter=JsCode(label_formatter)),
         tooltip_opts=opts.TooltipOpts(formatter=JsCode(tooltip_formatter)),
        )
    .set_global_opts(title_opts=opts.TitleOpts(title="美股树形图"))
)

treemap.render_notebook()

---
### 总结

生成直角坐标系的单图对象的代码通常如下：

	obj = ( Object(...).add_xaxis(...)
                       .add_yaxis(...)                                       
        	 	      .set_global_options(...) )

其中 `Object` 是直角坐标系中的常见元件，如 `Kline`, `Line` 和 `Bar` 等等,省略号 ... 代表各种配置。


生成非直角坐标系的单图对象的代码通常如下：

	obj = ( Object(...).add(...)
        	 	      .set_global_options(...) )

其中 `Object` 是非直角坐标系中的常见元件，如 `Pie`, `WordCount` 和 `TreeMap` 等等，省略号 ... 代表各种配置。

图对象生成后，显示它有两种方式：

	obj.render_notebook()	在 Jupyter Notebook 中显示
	obj.render('xxx.html')  在网页中显示

# <font color='#dc2624' face='微软雅黑'>4. 多图</font><a name='4'></a>
[<font color='black' face='微软雅黑'>回到目录</font>](#toc)

**`PyEcharts`** 中的多图有四种，并行图 (grid)、时间轮播图 (timeline)、选项卡图 (tab) 和页面图 (page)。

### <font color='#2b4750' face='微软雅黑'>4.1 并行图</font><a name='4.1'></a>
[<font color='black' face='微软雅黑'>回到章首</font>](#4)

用苹果股票、标准普尔 500 指数 (SPX) 和恐慌指数 (VIX) 数据举例，画出同一时段内的股票 K 线图和指数折线图。

In [59]:
stock_data = pd.read_csv( '1Y Stock Data.csv', parse_dates=[0], dayfirst=True )
stock_data.head().append(stock_data.tail())

,Date,Symbol,Open,High,Low,Close,Adj Close,Volume
0,2018-02-26,AAPL,176.350006,179.389999,176.210007,178.970001,176.285675,38162200
1,2018-02-27,AAPL,179.100006,180.479996,178.160004,178.389999,175.714386,38928100
2,2018-02-28,AAPL,179.259995,180.619995,178.050003,178.119995,175.448410,37782100
3,2018-03-01,AAPL,178.539993,179.779999,172.660004,175.000000,172.375214,48802000
4,2018-03-02,AAPL,172.800003,176.300003,172.449997,176.210007,173.567078,38454000
1255,2019-02-20,GS,198.729996,199.300003,197.509995,198.600006,198.600006,2266000
1256,2019-02-21,GS,198.970001,199.449997,195.050003,196.360001,196.360001,2785900
1257,2019-02-22,GS,196.600006,197.750000,195.199997,196.000000,196.000000,2626600
1258,2019-02-25,GS,198.000000,201.500000,197.710007,198.649994,198.649994,3032200
1259,2019-02-26,GS,198.470001,200.559998,196.550003,198.899994,198.899994,2498000


In [60]:
data = pd.read_csv( 'S&P500.csv', index_col=0, parse_dates=True, dayfirst=True )
spx = data[['Adj Close']].loc['2018-02-26':'2019-02-26']
spx.head(3).append(spx.tail(3))

,Adj Close
Date,
2018-02-26,2779.600098
2018-02-27,2744.280029
2018-02-28,2713.830078
2019-02-22,2792.669922
2019-02-25,2796.110107
2019-02-26,2793.899902


In [61]:
data = pd.read_csv( 'VIX.csv', index_col=0, parse_dates=True, dayfirst=True )
vix = data[['Adj Close']].loc['2018-02-26':'2019-02-26']
vix.head(3).append(vix.tail(3))

,Adj Close
Date,
2018-02-26,15.80
2018-02-27,18.59
2018-02-28,19.85
2019-02-22,13.51
2019-02-25,14.85
2019-02-26,15.17


In [66]:
code = 'AAPL'
stock = stock_data[ stock_data['Symbol']==code ]

date = stock['Date'].dt.strftime('%d-%b-%Y').tolist()
price = stock[['Open','Close','Low','High']].values.tolist()

kline = (
    Kline()
    .add_xaxis(date)
    .add_yaxis('', price)
    .set_global_opts(
        yaxis_opts=opts.AxisOpts(
            is_scale=True,
            splitarea_opts=opts.SplitAreaOpts(
                is_show=True, 
                areastyle_opts=opts.AreaStyleOpts(opacity=1)
            ),
        ),
        xaxis_opts=opts.AxisOpts(is_scale=True),
        title_opts=opts.TitleOpts(title=code, pos_top='25%'),
        datazoom_opts=[opts.DataZoomOpts(type_='slider',
                                         xaxis_index=[0, 1, 2],
                                         range_start=0,
                                         range_end=len(date),),
                       opts.DataZoomOpts(type_='inside',
                                         xaxis_index=[0, 1, 2],
                                         range_start=0,
                                         range_end=len(date),)],
    )
)

line1 = (
    Line()
    .add_xaxis(date)
    .add_yaxis(
            series_name='',
            y_axis=spx.values.tolist(),
            markpoint_opts=opts.MarkPointOpts(
                data=[opts.MarkPointItem(type_='min')]),
            is_smooth=True,
            is_hover_animation=False,
            linestyle_opts=opts.LineStyleOpts(width=3, opacity=0.5),
            label_opts=opts.LabelOpts(is_show=False),
    )
    .set_global_opts(
        xaxis_opts=opts.AxisOpts(is_scale=True),
        yaxis_opts=opts.AxisOpts(is_scale=True),
        title_opts=opts.TitleOpts(title='SPX', pos_top='55%'),
    )
)

line2 = (
    Line()
    .add_xaxis(date)
    .add_yaxis(
            series_name='',
            y_axis=vix.values.tolist(),
            markpoint_opts=opts.MarkPointOpts(
                data=[opts.MarkPointItem(type_='max')]),
            is_smooth=True,
            is_hover_animation=False,
            linestyle_opts=opts.LineStyleOpts(width=3, opacity=0.5),
            label_opts=opts.LabelOpts(is_show=False),
    )
    .set_global_opts(
        xaxis_opts=opts.AxisOpts(is_scale=True),
        yaxis_opts=opts.AxisOpts(is_scale=True),
        title_opts=opts.TitleOpts(title='VIX', pos_top='75%'),
    )
)

grid_chart = Grid( init_opts=opts.InitOpts(width="1000px", height="600px") )

grid_chart.add(
    kline,
    grid_opts=opts.GridOpts(pos_left="10%", pos_right="8%", pos_top="5%", height="40%")
)
grid_chart.add(
    line1,
    grid_opts=opts.GridOpts(pos_left="10%", pos_right="8%", pos_top="50%", height="15%")
)
grid_chart.add(
    line2,
    grid_opts=opts.GridOpts(pos_left="10%", pos_right="8%", pos_top="70%", height="15%")
)

grid_chart.render_notebook()

再看一个更酷的例子，模仿炒股软件画出 K 线图和交易量柱状图。在本例中使用标准普尔 500 指数 (SPX) 在 2018-02-26 到 2019-02-26 的数据。

In [67]:
data = pd.read_csv( 'S&P500.csv', index_col=0, parse_dates=True, dayfirst=True )
data = data.loc['2018-02-26':'2019-02-26']
data.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-02-26,2757.370117,2780.639893,2753.780029,2779.600098,2779.600098,3424650000
2018-02-27,2780.449951,2789.149902,2744.219971,2744.280029,2744.280029,3745080000
2018-02-28,2753.780029,2761.520020,2713.540039,2713.830078,2713.830078,4230660000
2018-03-01,2715.219971,2730.889893,2659.649902,2677.669922,2677.669922,4503970000
2018-03-02,2658.889893,2696.250000,2647.320068,2691.250000,2691.250000,3882450000


In [68]:
date = pd.to_datetime(data.index).strftime('%Y/%m/%d').tolist()
price = data[['Open','Close','Low','High']].values.tolist()
close = data['Close']
volume = data['Volume'].values.tolist()

In [82]:
kline = (
    Kline()
    .add_xaxis(xaxis_data=date)
    .add_yaxis(
        series_name="S&P index",
        y_axis=price,
        itemstyle_opts=opts.ItemStyleOpts(color="#ec0000", color0="#00da3c", 
                                          border_color="#ec0000", border_color0="#00da3c"),
    )
    .set_global_opts(
        title_opts=opts.TitleOpts(
            title="标普 500 指数",
            subtitle="MA5, MA20",
        ),
        xaxis_opts=opts.AxisOpts(type_="category"),
        yaxis_opts=opts.AxisOpts(
            is_scale=True,
            splitarea_opts=opts.SplitAreaOpts(
                is_show=True, areastyle_opts=opts.AreaStyleOpts(opacity=1)
            ),
        ),
        legend_opts=opts.LegendOpts(is_show=False),
        datazoom_opts=[
            opts.DataZoomOpts(
                is_show=False,
                type_="inside",
                xaxis_index=[0, 1],
                range_start=0,
                range_end=100,
            ),
            opts.DataZoomOpts(
                is_show=True,
                xaxis_index=[0, 1],
                type_="slider",
                pos_top="90%",
                range_start=0,
                range_end=100,
            ),
        ],
        tooltip_opts=opts.TooltipOpts(
            trigger="axis",
            axis_pointer_type="cross",
            background_color="rgba(245, 245, 245, 0.8)",
            border_width=1,
            border_color="#ccc",
            textstyle_opts=opts.TextStyleOpts(color="#000", font_size=10),
        ),
        visualmap_opts=opts.VisualMapOpts(
            is_show=False,
            dimension=2,
            series_index=3,
            is_piecewise=True,
            pieces=[
                {"value": -1, "color": "#ec0000"},
                {"value": 1, "color": "#00da3c"},
            ],
        ),
        axispointer_opts=opts.AxisPointerOpts(
            is_show=True,
            link=[{"xAxisIndex": "all"}],
            label=opts.LabelOpts(background_color="#777"),
        ),
        brush_opts=opts.BrushOpts(
            x_axis_index="all",
            brush_link="all",
            out_of_brush={"colorAlpha": 0.1},
            brush_type="lineX",
            brush_style = {"borderWidth": 1, "color":"rgba(120,140,180,0.3)", "borderColor":"black"},
        ),
    )
)

line = (
    Line()
    .add_xaxis(xaxis_data=date)
    .add_yaxis(
        series_name="MA5",
        y_axis=close.rolling(5).mean().tolist(),
        is_smooth=True,
        is_hover_animation=False,
        linestyle_opts=opts.LineStyleOpts(width=2, opacity=0.5),
        label_opts=opts.LabelOpts(is_show=False),
    )
    .add_yaxis(
        series_name="MA20",
        y_axis=close.rolling(20).mean().tolist(),
        is_smooth=True,
        is_hover_animation=False,
        linestyle_opts=opts.LineStyleOpts(width=2, opacity=0.5),
        label_opts=opts.LabelOpts(is_show=False),
    )
    .set_global_opts(xaxis_opts=opts.AxisOpts(type_="category"))
    )

bar = (
    Bar()
    .add_xaxis(xaxis_data=date)
    .add_yaxis(
        series_name="Volume",
        y_axis=[
            [i, volume[i], 1 if price[i][0] > price[i][1] else -1]
            for i in range(len(data))
        ],
        xaxis_index=1,
        yaxis_index=1,
        label_opts=opts.LabelOpts(is_show=False),
    )
    .set_global_opts(
        xaxis_opts=opts.AxisOpts(
            type_="category",
            is_scale=True,
            grid_index=1,
            boundary_gap=False,
            axisline_opts=opts.AxisLineOpts(is_on_zero=False),
            axistick_opts=opts.AxisTickOpts(is_show=False),
            splitline_opts=opts.SplitLineOpts(is_show=False),
            axislabel_opts=opts.LabelOpts(is_show=False),
            split_number=20,
            min_="dataMin",
            max_="dataMax",
        ),
        yaxis_opts=opts.AxisOpts(
            grid_index=1,
            is_scale=True,
            split_number=2,
        ),
        legend_opts=opts.LegendOpts(is_show=False),
    )
)

# Kline And Line
overlap_kline_line = kline.overlap(line)

# Grid Overlap + Bar
grid_chart = Grid( init_opts=opts.InitOpts(width="1000px", height="600px") )

grid_chart.add(
    overlap_kline_line,
    grid_opts=opts.GridOpts(pos_left="10%", pos_right="8%", height="50%")
    )
grid_chart.add(
    bar,
    grid_opts=opts.GridOpts(pos_left="10%", pos_right="8%", pos_top="70%", height="16%")
)

grid_chart.render_notebook()

### <font color='#2b4750' face='微软雅黑'>4.2 时间轮播图</font><a name='4.2'></a>
[<font color='black' face='微软雅黑'>回到章首</font>](#4)

本例想画出不同时点 (每月月底) 时五只股票 (苹果、阿里巴巴、京东、脸书和高盛) 交易额 (价格×交易量) 的成分图。用 `Matplotlib` 则需要画 n 张饼状图，其中 n 为月份个数，但用 **`PyEcharts`** 里面的 `Timeline` 模块，可画出随时间线轮播动态图，一图可包含 n 张饼状图。首先读取包含五只股票的数据，做以下三个操作，

- 保留 Date, Symbol, Adj Close 和 Volume 几列，最后两列是股票的收盘价和交易量 (用于计算交易额)
- 从 Date 中获取 Year 和 Month 的信息并插入表中，用于根据年份和月份来分组。
- 按 [Symbol, Year, Month] 分组，获取每只股票月底的数据

In [86]:
stock_data = pd.read_csv( '1Y Stock Data.csv', parse_dates=[0], dayfirst=True )

data = stock_data[['Date', 'Symbol', 'Adj Close', 'Volume']]
data.insert( 1, 'Year', pd.DatetimeIndex(data['Date']).year )
data.insert( 2, 'Month', pd.DatetimeIndex(data['Date']).month )
data.head(3).append(data.tail(3))

,Date,Year,Month,Symbol,Adj Close,Volume
0,2018-02-26,2018,2,AAPL,176.285675,38162200
1,2018-02-27,2018,2,AAPL,175.714386,38928100
2,2018-02-28,2018,2,AAPL,175.448410,37782100
1257,2019-02-22,2019,2,GS,196.000000,2626600
1258,2019-02-25,2019,2,GS,198.649994,3032200
1259,2019-02-26,2019,2,GS,198.899994,2498000


In [87]:
def print_groups( group_obj ):
    for name, group in group_obj:
        print( name )
        print( group.head() )

In [88]:
grouped = data.groupby(['Symbol', 'Year', 'Month'])
print_groups(grouped)

('AAPL', 2018, 2)
        Date  Year  Month Symbol   Adj Close    Volume
0 2018-02-26  2018      2   AAPL  176.285675  38162200
1 2018-02-27  2018      2   AAPL  175.714386  38928100
2 2018-02-28  2018      2   AAPL  175.448410  37782100
('AAPL', 2018, 3)
        Date  Year  Month Symbol   Adj Close    Volume
3 2018-03-01  2018      3   AAPL  172.375214  48802000
4 2018-03-02  2018      3   AAPL  173.567078  38454000
5 2018-03-05  2018      3   AAPL  174.167923  28401400
6 2018-03-06  2018      3   AAPL  174.020172  23788500
7 2018-03-07  2018      3   AAPL  172.404770  31703500
('AAPL', 2018, 4)
         Date  Year  Month Symbol   Adj Close    Volume
24 2018-04-02  2018      4   AAPL  164.180008  37586800
25 2018-04-03  2018      4   AAPL  165.864349  30278000
26 2018-04-04  2018      4   AAPL  169.036072  34605500
27 2018-04-05  2018      4   AAPL  170.208206  26933200
28 2018-04-06  2018      4   AAPL  165.854523  35005300
('AAPL', 2018, 5)
         Date  Year  Month Symbol   Adj Cl

('FB', 2018, 2)
          Date  Year  Month Symbol   Adj Close    Volume
756 2018-02-26  2018      2     FB  184.929993  17599700
757 2018-02-27  2018      2     FB  181.460007  15849800
758 2018-02-28  2018      2     FB  178.320007  18783000
('FB', 2018, 3)
          Date  Year  Month Symbol   Adj Close    Volume
759 2018-03-01  2018      3     FB  175.940002  23201600
760 2018-03-02  2018      3     FB  176.619995  20025900
761 2018-03-05  2018      3     FB  180.399994  16189300
762 2018-03-06  2018      3     FB  179.779999  15086800
763 2018-03-07  2018      3     FB  183.710007  19097300
('FB', 2018, 4)
          Date  Year  Month Symbol   Adj Close    Volume
780 2018-04-02  2018      4     FB  155.389999  36796000
781 2018-04-03  2018      4     FB  156.110001  42543900
782 2018-04-04  2018      4     FB  155.100006  49885600
783 2018-04-05  2018      4     FB  159.339996  41449600
784 2018-04-06  2018      4     FB  157.199997  41644800
('FB', 2018, 5)
          Date  Year  Mo

1247 2019-02-07  2019      2     GS  193.070007  2948800
('JD', 2018, 2)
          Date  Year  Month Symbol  Adj Close    Volume
504 2018-02-26  2018      2     JD  48.799999   9126900
505 2018-02-27  2018      2     JD  47.040001  12688600
506 2018-02-28  2018      2     JD  47.150002  12227800
('JD', 2018, 3)
          Date  Year  Month Symbol  Adj Close    Volume
507 2018-03-01  2018      3     JD  46.209999  15883200
508 2018-03-02  2018      3     JD  43.799999  34897000
509 2018-03-05  2018      3     JD  42.990002  22091000
510 2018-03-06  2018      3     JD  43.139999  14014400
511 2018-03-07  2018      3     JD  44.410000  11848600
('JD', 2018, 4)
          Date  Year  Month Symbol  Adj Close    Volume
528 2018-04-02  2018      4     JD  39.080002  14019000
529 2018-04-03  2018      4     JD  38.980000   9496700
530 2018-04-04  2018      4     JD  39.439999  11933700
531 2018-04-05  2018      4     JD  40.189999  10002800
532 2018-04-06  2018      4     JD  39.230000  12637000

In [89]:
group = grouped.apply( lambda df: df.iloc[-1,:] )
group.head(3).append(group.tail(3))

Date  Year  Month Symbol   Adj Close    Volume
Symbol Year Month                                                     
AAPL   2018 2     2018-02-28  2018      2   AAPL  175.448410  37782100
            3     2018-03-29  2018      3   AAPL  165.263504  38398500
            4     2018-04-30  2018      4   AAPL  162.781296  42427400
JD     2018 12    2018-12-31  2018     12     JD   20.930000  19422400
       2019 1     2019-01-31  2019      1     JD   24.850000  19318400
            2     2019-02-26  2019      2     JD   26.590000  20264100

In [93]:
attribute = data.Symbol.unique()
n_stock = len(attribute)

MV = group['Adj Close'] * group['Volume']
value = np.reshape( MV.values, (n_stock,-1) ).T.tolist()
date = group['Date'].dt.strftime('%d-%b-%Y').unique()

tooltip_formatter = """function (params) {
        return '公司: ' + params.name  + 
            '<br>' + '市值: ' + window.parseFloat(params.value/100000000).toFixed(2) + ' 亿美元' +
            '<br>' + '占比: ' + params.percent + '%'
    }"""

timeline = Timeline()
for d, v in zip(date, value):
    pie = (
        Pie()
        .add(
            d,
            [list(z) for z in zip(attribute, v)],
            rosetype="radius",
            radius=["10%", "70%"],
            label_opts=opts.LabelOpts(
                # {a}（系列名称），{b}（数据项名称），{c}（数值）, {d}（百分比）
                formatter= "日期: {a} \n 公司：{b} \n 市值：{c} \n 占比：{d}% ",
                background_color="#eee",
                border_color="grey",
                border_width=1,
                border_radius=5),
            tooltip_opts=opts.TooltipOpts(formatter=JsCode(tooltip_formatter)),
        )
        .set_global_opts(title_opts=opts.TitleOpts(f"{d} 时公司市值"))
    )

    timeline.add(pie, d)

timeline.render_notebook()

### <font color='#2b4750' face='微软雅黑'>4.3 选项卡图</font><a name='4.3'></a>
[<font color='black' face='微软雅黑'>回到章首</font>](#4)

选项卡式可将每张效果图进行全尺寸切换，而不必将其压缩到彼此相邻或重叠的位置。如果要展示的图多，选项卡图是最合适的形式。

定义函数 `Kline_func()` 返回 `Kline` 对象。

In [114]:
curr = 'EURUSD'
EURUSD = data_converter( currency_daily, curr, 'FX' )

def Kline_func(data, asset) -> Kline:
    date, price = data.index, data.values
    date_list = pd.to_datetime(date).strftime('%Y/%m/%d').tolist()

    kline = (
        Kline()
        .add_xaxis(date_list)
        .add_yaxis('', price.tolist())
        .set_global_opts(
            xaxis_opts=opts.AxisOpts(is_scale=True),
            yaxis_opts=opts.AxisOpts(
                is_scale=True,
                splitarea_opts=opts.SplitAreaOpts(
                    is_show=True, 
                    areastyle_opts=opts.AreaStyleOpts(opacity=0.5)
                ),
            ),
            title_opts=opts.TitleOpts(title=asset +' Chart'),
            datazoom_opts=[opts.DataZoomOpts(type_='inside')],
        )
    )
    return kline

定义函数 `Grid_func0()` 返回 `Grid` 对象。

In [115]:
stock_data = pd.read_csv( '1Y Stock Data.csv', parse_dates=[0], dayfirst=True )
stock_data.head().append(stock_data.tail())

data = pd.read_csv( 'S&P500.csv', index_col=0, parse_dates=True, dayfirst=True )
spx = data[['Adj Close']].loc['2018-02-26':'2019-02-26']

data = pd.read_csv( 'VIX.csv', index_col=0, parse_dates=True, dayfirst=True )
vix = data[['Adj Close']].loc['2018-02-26':'2019-02-26']

code = 'AAPL'

def Grid_func0(stock, code, index, vol) -> Grid:

    stock = stock[ stock['Symbol']==code ]

    date = stock['Date'].dt.strftime('%d-%b-%Y').tolist()
    price = stock[['Open','Close','Low','High']].values.tolist()

    kline = (
        Kline()
        .add_xaxis(date)
        .add_yaxis('', price)
        .set_global_opts(
            yaxis_opts=opts.AxisOpts(
                is_scale=True,
                splitarea_opts=opts.SplitAreaOpts(
                    is_show=True, 
                    areastyle_opts=opts.AreaStyleOpts(opacity=1)
                ),
            ),
            xaxis_opts=opts.AxisOpts(is_scale=True),
            title_opts=opts.TitleOpts(title=code, pos_top='25%'),
            datazoom_opts=[opts.DataZoomOpts(type_='slider',
                                             xaxis_index=[0, 1, 2],
                                             range_start=0,
                                             range_end=len(date),),
                           opts.DataZoomOpts(type_='inside',
                                             xaxis_index=[0, 1, 2],
                                             range_start=0,
                                             range_end=len(date),)],
        )
    )

    line1 = (
        Line()
        .add_xaxis(date)
        .add_yaxis(
                series_name='',
                y_axis=index.values.tolist(),
                markpoint_opts=opts.MarkPointOpts(
                    data=[opts.MarkPointItem(type_='min')]),
                is_smooth=True,
                is_hover_animation=False,
                linestyle_opts=opts.LineStyleOpts(width=3, opacity=0.5),
                label_opts=opts.LabelOpts(is_show=False),
        )
        .set_global_opts(
            xaxis_opts=opts.AxisOpts(is_scale=True),
            yaxis_opts=opts.AxisOpts(is_scale=True),
            title_opts=opts.TitleOpts(title='SPX', pos_top='55%'),
        )
    )

    line2 = (
        Line()
        .add_xaxis(date)
        .add_yaxis(
                series_name='',
                y_axis=vol.values.tolist(),
                markpoint_opts=opts.MarkPointOpts(
                    data=[opts.MarkPointItem(type_='max')]),
                is_smooth=True,
                is_hover_animation=False,
                linestyle_opts=opts.LineStyleOpts(width=3, opacity=0.5),
                label_opts=opts.LabelOpts(is_show=False),
        )
        .set_global_opts(
            xaxis_opts=opts.AxisOpts(is_scale=True),
            yaxis_opts=opts.AxisOpts(is_scale=True),
            title_opts=opts.TitleOpts(title='VIX', pos_top='75%'),
        )
    )

    grid_chart = Grid( init_opts=opts.InitOpts(width="1000px", height="600px") )

    grid_chart.add(
        kline,
        grid_opts=opts.GridOpts(pos_left="10%", pos_right="8%", pos_top="5%", height="40%")
    )
    grid_chart.add(
        line1,
        grid_opts=opts.GridOpts(pos_left="10%", pos_right="8%", pos_top="50%", height="15%")
    )
    grid_chart.add(
        line2,
        grid_opts=opts.GridOpts(pos_left="10%", pos_right="8%", pos_top="70%", height="15%")
    )

    return grid_chart

定义函数 `Grid_func()` 返回 `Grid` 对象。

In [116]:
SP500 = pd.read_csv( 'S&P500.csv', index_col=0, parse_dates=True, dayfirst=True )
SP500 = SP500.loc['2018-02-26':'2019-02-26']

def Grid_func(data) -> Grid:

    date = pd.to_datetime(data.index).strftime('%Y/%m/%d').tolist()
    price = data[['Open','Close','Low','High']].values.tolist()
    close = data['Close']
    volume = data['Volume'].values.tolist()

    kline = (
        Kline()
        .add_xaxis(xaxis_data=date)
        .add_yaxis(
            series_name="S&P index",
            y_axis=price,
            itemstyle_opts=opts.ItemStyleOpts(color="#ec0000", color0="#00da3c", 
                                              border_color="#ec0000", border_color0="#00da3c"),
        )
        .set_global_opts(
            title_opts=opts.TitleOpts(
                title="标普 500 指数",
                subtitle="MA5, MA20",
            ),
            xaxis_opts=opts.AxisOpts(type_="category"),
            yaxis_opts=opts.AxisOpts(
                is_scale=True,
                splitarea_opts=opts.SplitAreaOpts(
                    is_show=True, areastyle_opts=opts.AreaStyleOpts(opacity=1)
                ),
            ),
            legend_opts=opts.LegendOpts(is_show=False),
            datazoom_opts=[
                opts.DataZoomOpts(
                    is_show=False,
                    type_="inside",
                    xaxis_index=[0, 1],
                    range_start=0,
                    range_end=100,
                ),
                opts.DataZoomOpts(
                    is_show=True,
                    xaxis_index=[0, 1],
                    type_="slider",
                    pos_top="90%",
                    range_start=0,
                    range_end=100,
                ),
            ],
            tooltip_opts=opts.TooltipOpts(
                trigger="axis",
                axis_pointer_type="cross",
                background_color="rgba(245, 245, 245, 0.8)",
                border_width=1,
                border_color="#ccc",
                textstyle_opts=opts.TextStyleOpts(color="#000", font_size=10),
                #formatter=JsCode(js_code_str),
            ),
            visualmap_opts=opts.VisualMapOpts(
                is_show=False,
                dimension=2,
                series_index=3,
                is_piecewise=True,
                pieces=[
                    {"value": -1, "color": "#ec0000"},
                    {"value": 1, "color": "#00da3c"},
                ],
            ),
            axispointer_opts=opts.AxisPointerOpts(
                is_show=True,
                link=[{"xAxisIndex": "all"}],
                label=opts.LabelOpts(background_color="#777"),
            ),
            brush_opts=opts.BrushOpts(
                x_axis_index="all",
                brush_link="all",
                out_of_brush={"colorAlpha": 0.1},
                brush_type="lineX",
            ),
        )
    )

    line = (
        Line()
        .add_xaxis(xaxis_data=date)
        .add_yaxis(
            series_name="MA5",
            y_axis=close.rolling(5).mean().tolist(),
            is_smooth=True,
            is_hover_animation=False,
            linestyle_opts=opts.LineStyleOpts(width=2, opacity=0.5),
            label_opts=opts.LabelOpts(is_show=False),
        )
        .add_yaxis(
            series_name="MA20",
            y_axis=close.rolling(20).mean().tolist(),
            is_smooth=True,
            is_hover_animation=False,
            linestyle_opts=opts.LineStyleOpts(width=2, opacity=0.5),
            label_opts=opts.LabelOpts(is_show=False),
        )
        .set_global_opts(xaxis_opts=opts.AxisOpts(type_="category"))
        )

    bar = (
        Bar()
        .add_xaxis(xaxis_data=date)
        .add_yaxis(
            series_name="Volume",
            y_axis=[
                [i, volume[i], 1 if price[i][0] > price[i][1] else -1]
                for i in range(len(data))
            ],
            xaxis_index=1,
            yaxis_index=1,
            label_opts=opts.LabelOpts(is_show=False),
        )
        .set_global_opts(
            xaxis_opts=opts.AxisOpts(
                type_="category",
                is_scale=True,
                grid_index=1,
                boundary_gap=False,
                axisline_opts=opts.AxisLineOpts(is_on_zero=False),
                axistick_opts=opts.AxisTickOpts(is_show=False),
                splitline_opts=opts.SplitLineOpts(is_show=False),
                axislabel_opts=opts.LabelOpts(is_show=False),
                split_number=20,
                min_="dataMin",
                max_="dataMax",
            ),
            yaxis_opts=opts.AxisOpts(
                grid_index=1,
                is_scale=True,
                split_number=2,
            ),
            legend_opts=opts.LegendOpts(is_show=False),
        )
    )

    # Kline And Line
    overlap_kline_line = kline.overlap(line)

    # Grid Overlap + Bar
    grid_chart = Grid( init_opts=opts.InitOpts(width="1000px", height="600px") )

    grid_chart.add(
        overlap_kline_line,
        grid_opts=opts.GridOpts(pos_left="10%", pos_right="8%", height="50%")
        )
    grid_chart.add(
        bar,
        grid_opts=opts.GridOpts(pos_left="10%", pos_right="8%", pos_top="70%", height="16%")
    )

    return grid_chart

定义函数 `Timeline_func()` 返回 `Timeline` 对象。

In [117]:
data_1Y = pd.read_csv( '1Y Stock Data.csv', parse_dates=[0], dayfirst=True )
data = data_1Y[['Date', 'Symbol', 'Adj Close', 'Volume']]
data.insert( 1, 'Year', pd.DatetimeIndex(data['Date']).year )
data.insert( 2, 'Month', pd.DatetimeIndex(data['Date']).month )
grouped = data.groupby(['Symbol', 'Year', 'Month'])
group_data = grouped.apply( lambda df: df.iloc[-1,:] )

def Timeline_func(data) -> Timeline:

    attribute = set([idx[0] for idx in group_data.index])
    n_stock = len(attribute)

    MV = group['Adj Close'] * group['Volume']
    value = np.reshape( MV.values, (n_stock,-1) ).T.tolist()
    date = group['Date'].dt.strftime('%d-%b-%Y').unique()

    tooltip_formatter = """function (params) {
            return '公司: ' + params.name  + 
                '<br>' + '市值: ' + window.parseFloat(params.value/100000000).toFixed(2) + ' 亿美元' +
                '<br>' + '占比: ' + params.percent + '%'
        }"""

    timeline = Timeline()
    for d, v in zip(date, value):
        pie = (
            Pie()
            .add(
                "",
                [list(z) for z in zip(attribute, v)],
                rosetype="radius",
                radius=["10%", "70%"],
                label_opts=opts.LabelOpts(
                    formatter= " 公司：{b} \n 市值：{c} \n 占比：{d}% ",
                    background_color="#eee",
                    border_color="grey",
                    border_width=1,
                    border_radius=5),
                tooltip_opts=opts.TooltipOpts(formatter=JsCode(tooltip_formatter)),
            )
            .set_global_opts(title_opts=opts.TitleOpts(f"{d} 时公司市值"))
        )

        timeline.add(pie, d)
        
    return timeline

定义函数 `TreeMap_func()` 返回 `TreeMap` 对象。

In [118]:
data_US = pd.read_csv( 'US Stock Info.csv' )

def TreeMap_func(data) -> TreeMap:
    
    grouped = data.groupby(['sector'])
    group_value = grouped.apply( lambda df: np.sum(df['market_cap']) )
    data_for_treemap = []

    for i, grp in enumerate(grouped):

        total_mktcap, sector = group_value[i], group_value.index[i]
        code, r, mktcap = grp[1]['code'], grp[1]['return'], grp[1]['market_cap']

        children = []

        for j in range(len(code)):

            j_data = { 'value' : mktcap.iloc[j], 
                       'name': [code.iloc[j], r.iloc[j], sector] }

            children.append( j_data )

        i_data = { 'value' : total_mktcap,
                   'name': sector,
                   'children': children }

        data_for_treemap.append( i_data )

    label_formatter = """function (params) {
            sign = (params.name[1] > 0) ? '+' : '';
            return params.name[0] + '\\n' + sign + window.parseFloat(params.name[1]*100).toFixed(2) + '%'
        }"""

    tooltip_formatter = """function (params) {
        up_down_flat = (params.name[1] > 0) ? '涨' : (params.name[1] < 0) ? '跌' : '平';
        return params.name[0] + '<br>' + '行业: ' + params.name[2]  + 
        '<br>' + '市值: ' + window.parseFloat(params.value/100000000) + ' 亿美元' +
        '<br>' + '涨跌: ' + up_down_flat;
    }"""

    treemap = (
        TreeMap( init_opts=opts.InitOpts(width="1000px", height="600px") )
        .add("", 
             data_for_treemap,
             label_opts=opts.LabelOpts(position="inside", font_size=10, formatter=JsCode(label_formatter)),
             tooltip_opts=opts.TooltipOpts(formatter=JsCode(tooltip_formatter)),
            )
        .set_global_opts(title_opts=opts.TitleOpts(title="美股树形图"))
    )
    return treemap

定义函数 `WordCloud_func()` 返回 `WordCloud` 对象。

In [119]:
def WordCloud_func(attr, count) -> WordCloud:
    wordcloud = (
        WordCloud()
        .add(series_name="",
             data_pair=[pair for pair in zip(attr,count)], 
             word_size_range=[20, 100]
            )
        .set_global_opts(
            title_opts=opts.TitleOpts(
                title="王的机器用户分析", title_textstyle_opts=opts.TextStyleOpts(font_size=23)
            ),
            tooltip_opts=opts.TooltipOpts(is_show=True),
        )
    )
    return wordcloud

创建完 `Tab` 对象后，按顺序使用多个 `add()` 函数添加子图就行了，每个子图就是上面每个例子中定义的绘图函数。

In [120]:
tab = Tab()
tab.add(Kline_func(EURUSD, curr), "外汇 K 线图")
tab.add(Grid_func0(stock_data, "AAPL", spx, vix) , "苹果+SP500+VIX 多图")
tab.add(Grid_func(SP500), "标普 500 指数量价图")
tab.add(Timeline_func(group_data), "时间轮播图")
tab.add(TreeMap_func(data_US), "美股树形表图")
tab.add(WordCloud_func(attr, count), "王的机器用户图")
tab.render_notebook()

### <font color='#2b4750' face='微软雅黑'>4.4 页面图</font><a name='4.4'></a>
[<font color='black' face='微软雅黑'>回到章首</font>](#4)

选项卡图是通过选标签来展示多图，顺序图是按顺序展示多图，包含三种布局方式：

1. DefaultPageLayout
2. SimplePageLayout
3. DraggablePageLayout

下面代码用 `SimplePageLayout` 来举例。代码非常易懂。创建完 `Page` 对象后，在一个 `add()` 函数中按顺序添加子图就行了。注意在选项卡图是分别用 `add()` 函数来添加子图的。两者的差异如下：

    选项卡图	Tab().add( chart_1 ).add( chart_2 ). … .add( chart_n )
    顺序图 	 Page().add( chart_1, chart_2, …, chart_n )

In [46]:
page = Page(layout=Page.SimplePageLayout)
page.add( Kline_func(EURUSD, curr), 
          Timeline_func(group_data), 
          TreeMap_func(data_US), 
          WordCloud_func(attr, count) )
page.render_notebook()